In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import textwrap
from sklearn.metrics import roc_auc_score

In [6]:
def normalize_label(label):
    """
    Normalize prediction labels to consistent format.
    """
    if pd.isna(label):
        return 'unknown'
    
    label_str = str(label).lower().strip()
    
    if 'human' in label_str:
        return 'human'
    elif 'ai' in label_str or 'generated' in label_str:
        return 'ai'
    else:
        return 'unknown'

def reorganize_meta_param_csvs(input_files):
    """
    Reorganize multiple meta-parameter CSVs into a single clean DataFrame.

    Parameters:
    - input_files: list of CSV file paths

    Returns:
    - df_clean: reorganized combined DataFrame
    """
    reorganized_data = []

    for file in input_files:
        # Use quoting=1 (QUOTE_ALL) or quoting=3 (QUOTE_MINIMAL) to handle quoted fields properly
        # on_bad_lines='skip' helps with malformed rows
        df = pd.read_csv(file, encoding='utf8', quotechar='"', escapechar=None, on_bad_lines='warn')
        
        # Check for any rows where key columns are NaN (indicates parsing issues)
        missing_data = df[df['question'].isna() | df['answer'].isna()]
        if len(missing_data) > 0:
            print(f"Warning: {len(missing_data)} rows have missing question or answer data")
        
        # Iterate through each row once
        for idx, row in df.iterrows():
            # Human answer
            reorganized_data.append({
                'question': row['question'],
                'answer': row['answer'],
                'true_label': 'human',
                'predicted_label': normalize_label(row['answer_detection_prediction']),
                'score': row['answer_detection_score'],
                'temperature': row['temperature'],
                'top_p': row['top_p'],
                'top_k': row['top_k'],
                'repeat_penalty': row['repeat_penalty']
            })
            
            # AI answer
            reorganized_data.append({
                'question': row['question'],
                'answer': row['question_answer_ai'],
                'true_label': 'ai',
                'predicted_label': normalize_label(row['question_answer_ai_detection_prediction']),
                'score': row['question_answer_ai_detection_score'],
                'temperature': row['temperature'],
                'top_p': row['top_p'],
                'top_k': row['top_k'],
                'repeat_penalty': row['repeat_penalty']
            })

    df_clean = pd.DataFrame(reorganized_data)
    
    return df_clean
    
    
file_list = [
    # "datasets/output_param_20_binoculars_default.csv",
    "datasets/output_param_20_binoculars_temperature0.0.csv",
    "datasets/output_param_20_binoculars_top_k5.csv",
    "datasets/output_param_20_binoculars_top_p0.6.csv",
    "datasets/output_param_20_binoculars_repeat_penalty1.0.csv",
]
df_clean = reorganize_meta_param_csvs(file_list)
print(df_clean)

                                              question  \
0    Postgresql allows adding comments to objects s...   
1    Postgresql allows adding comments to objects s...   
2    I was doing some work in scipy and a conversat...   
3    I was doing some work in scipy and a conversat...   
4    Can a permutation matrix ($P$) be used to chan...   
..                                                 ...   
715  Let $x,y,z$ be positive real numbers such that...   
716  Some people say that it's awful that humans ea...   
717  Some people say that it's awful that humans ea...   
718  I just started working at a company that doesn...   
719  I just started working at a company that doesn...   

                                                answer true_label  \
0    All comments are stored in pg_description To g...      human   
1    You can use the `pg_catalog.obj_description` f...         ai   
2    Let the CDF $F$ equal $1-1/n$ at the integers ...      human   
3    1. Consider the random

In [7]:
df_clean['true_binary'] = df_clean['true_label'].map({'human': 1, 'ai': 0})

# Group by metaparameters
grouped = df_clean.groupby(['temperature', 'top_p', 'top_k', 'repeat_penalty'])

# Calculate AUROC for each group
results = []
for name, group in grouped:
    try:
        auroc = roc_auc_score(group['true_binary'], group['score'])
    except ValueError:
        auroc = None  # Handle cases with only one class
    results.append({
        'temperature': name[0],
        'top_p': name[1],
        'top_k': name[2],
        'repeat_penalty': name[3],
        'AUROC': auroc
    })

# Convert results to a dataframe
auroc_results = pd.DataFrame(results)
auroc_results[auroc_results['repeat_penalty'] != 1.1]


,temperature,top_p,top_k,repeat_penalty,AUROC
7,0.8,0.95,40,1.0,0.9475
8,0.8,0.95,40,1.2,0.8125
9,0.8,0.95,40,1.3,0.6625
10,0.8,0.95,40,1.4,0.6775
11,0.8,0.95,40,1.5,0.5850
12,0.8,0.95,40,1.6,0.4925
